# Demographics and Employment in the US
#### By John Bobo based on a problem set from MIT’s Analytics Edge MOOC
#### April 28, 2016

In the wake of the Great Recession of 2009, there has been a good deal of focus on employment statistics, one of the most important metrics policymakers use to gauge the overall strength of the economy. In the United States, the government measures unemployment using the Current Population Survey (CPS), which collects demographic and employment information from a wide range of Americans each month. In this exercise, we will employ the topics reviewed in the lectures as well as a few new techniques using the September 2013 version of this rich, nationally representative dataset (available [online](http://thedataweb.rm.census.gov/ftp/cps_ftp.html)).

The observations in the dataset represent people surveyed in the September 2013 CPS who actually completed a survey. While the full dataset has 385 variables, in this exercise we will use a more compact version of the dataset, [CPSData.csv](https://d37djvu3ytnwxt.cloudfront.net/asset-v1:MITx+15.071x_3+1T2016+type@asset+block/CPSData.csv), which has the following variables:

  * **PeopleInHousehold**: The number of people in the interviewee's household.

  * **Region**: The census region where the interviewee lives.

  * **State**: The state where the interviewee lives.

  * **MetroAreaCode**: A code that identifies the metropolitan area in which the interviewee lives (missing if the interviewee does not live in a metropolitan area). The mapping from codes to names of metropolitan areas is provided in the file [MetroAreaCodes.csv](https://d37djvu3ytnwxt.cloudfront.net/asset-v1:MITx+15.071x_3+1T2016+type@asset+block/MetroAreaCodes.csv).

  * **Age**: The age, in years, of the interviewee. 80 represents people aged 80-84, and 85 represents people aged 85 and higher.

  * **Married**: The marriage status of the interviewee.

  * **Sex**: The sex of the interviewee.

  * **Education**: The maximum level of education obtained by the interviewee.

  * **Race**: The race of the interviewee.

  * **Hispanic**: Whether the interviewee is of Hispanic ethnicity.

  * **CountryOfBirthCode**: A code identifying the country of birth of the interviewee. The mapping from codes to names of countries is provided in the file [CountryCodes.csv](https://d37djvu3ytnwxt.cloudfront.net/asset-v1:MITx+15.071x_3+1T2016+type@asset+block/CountryCodes.csv).

  * **Citizenship**: The United States citizenship status of the interviewee.

  * **EmploymentStatus**: The status of employment of the interviewee.

  * **Industry**: The industry of employment of the interviewee (only available if they are employed).
  
***
  
## Loading and Summarizing the Dataset

In [1]:
import numpy as np
import pandas as pd

in_file = '/Users/johnbobo/analytics_edge/data/CPSData.csv'
cps = pd.read_csv(in_file)

In [2]:
cps.describe()

,PeopleInHousehold,MetroAreaCode,Age,Hispanic,CountryOfBirthCode
count,131302.000000,97064.000000,131302.000000,131302.000000,131302.000000
mean,3.284276,35074.709264,38.829226,0.139328,82.684079
std,1.700173,16551.637597,22.897130,0.346289,75.476511
min,1.000000,10420.000000,0.000000,0.000000,57.000000
25%,2.000000,21780.000000,19.000000,0.000000,57.000000
50%,3.000000,34740.000000,39.000000,0.000000,57.000000
75%,4.000000,41860.000000,57.000000,0.000000,57.000000
max,15.000000,79600.000000,85.000000,1.000000,555.000000


**How many interviewees are in the dataset?**  
**Answer**: 131302

***

**Among the interviewees with a value reported for the Industry variable (not NA nor `(Other)`), what is the most common industry of employment?**  

In [3]:
cps['Industry'].value_counts().head(1)

Educational and health services    15017
Name: Industry, dtype: int64

**Which state has the fewest interviewees?**

In [4]:
cps['State'].value_counts().tail(1)

New Mexico    1102
Name: State, dtype: int64

**Which state has the largest number of interviewees?**

In [5]:
cps['State'].value_counts().head(1)

California    11570
Name: State, dtype: int64

**What proportion of interviewees are citizens of the United States?**

In [6]:
np.mean(cps['Citizenship'].str.contains('^Citizen'))

0.94219433062710389

**The CPS differentiates between race (with possible values American Indian, Asian, Black, Pacific Islander, White, or Multiracial) and ethnicity. A number of interviewees are of Hispanic ethnicity, as captured by the Hispanic variable. For which races are there at least 250 interviewees in the CPS dataset of Hispanic ethnicity?** 

In [7]:
cps[cps.Hispanic == 1]['Race'].value_counts().head(4)

White              16731
Black                621
Multiracial          448
American Indian      304
Name: Race, dtype: int64

**Which variables have at least one interviewee with a missing (NA) value?**

In [8]:
cps.isnull().sum()

PeopleInHousehold         0
Region                    0
State                     0
MetroAreaCode         34238
Age                       0
Married               25338
Sex                       0
Education             25338
Race                      0
Hispanic                  0
CountryOfBirthCode        0
Citizenship               0
EmploymentStatus      25789
Industry              65060
dtype: int64

***

Often when evaluating a new dataset, we try to identify if there is a pattern in the missing values in the dataset. We will try to determine if there is a pattern in the missing values of the `Married` variable. We can see the breakdown of whether `Married` is missing based on the reported value of the `Age` variable:

In [9]:
cps.groupby('Age').agg({'Married': lambda x: x.isnull().sum()})

,Married
Age,
0,1283
1,1559
2,1574
3,1693
4,1695
5,1795
6,1721
7,1681
8,1729


**What can we see from these tables?**  
**Answer**: From these we can see the `Married` variable being missing is related to the `Age` value for the interviewee. Kids aren't really able to get married so this makes sense.

***

As mentioned in the variable descriptions, MetroAreaCode is missing if an interviewee does not live in a metropolitan area. Using the same technique as in the previous question, answer the following questions about people who live in non-metropolitan areas.

**How many states had all interviewees living in a non-metropolitan area (aka they have a missing MetroAreaCode value)? For this question, treat the District of Columbia as a state (even though it is not technically a state).**

In [10]:
cps.groupby('State').agg({'MetroAreaCode': lambda x: x.isnull().all()}).sum()

MetroAreaCode    2.0
dtype: float64

**How many states had all interviewees living in a metropolitan area? Again, treat the District of Columbia as a state.**  

In [11]:
cps.groupby('State').agg({'MetroAreaCode': lambda x: not x.isnull().any()}).sum()

MetroAreaCode    3.0
dtype: float64

**Which region of the United States has the largest proportion of interviewees living in a non-metropolitan area?**

In [12]:
cps.groupby('Region').agg({'MetroAreaCode': lambda x: np.mean(x.isnull())}).head(1)

,MetroAreaCode
Region,
Midwest,0.347869


**Which state has a proportion of interviewees living in a non-metropolitan area closest to 30%?**

In [13]:
cps.groupby('State').agg({'MetroAreaCode': lambda x: np.mean(x.isnull())}).sort_values('MetroAreaCode')

,MetroAreaCode
State,
Rhode Island,0.000000
New Jersey,0.000000
District of Columbia,0.000000
California,0.020484
Florida,0.039231
Massachusetts,0.064922
Maryland,0.069375
New York,0.080608
Connecticut,0.085684


**Answer:** Wisconsin

**Which state has the largest proportion of non-metropolitan interviewees, ignoring states where all interviewees were non-metropolitan?**  
**Answer**: Montana.

## Integrating Metropolitan Area Data
Codes like `MetroAreaCode` and `CountryOfBirthCode` are a compact way to encode factor variables with text as their possible values, and they are therefore quite common in survey datasets. In fact, all but one of the variables in this dataset were actually stored by a numeric code in the original CPS datafile.

When analyzing a variable stored by a numeric code, we will often want to convert it into the values the codes represent. To do this, we will use a dictionary, which maps the the code to the actual value of the variable. We have dictionaries `MetroAreaCodes.csv` and `CountryCodes.csv`, which respectively map `MetroAreaCode` and `CountryOfBirthCode` into their true values. Now we'll read these two dictionaries into data frames `metro_area_map` and `country_map` and merge them into our data frame.

In [14]:
metro_file = "/Users/johnbobo/analytics_edge/data/MetroAreaCodes.csv"
country_file = "/Users/johnbobo/analytics_edge/data/CountryCodes.csv"

metro_area_map = pd.read_csv(metro_file)
country_map = pd.read_csv(country_file)

df = pd.merge(cps, country_map, left_on='CountryOfBirthCode', right_on='Code', how='left')
df = pd.merge(df, metro_area_map, left_on='MetroAreaCode', right_on='Code', how='left')

**How many observations (codes for metropolitan areas) are there in MetroAreaMap?**

In [15]:
metro_area_map.shape[0]

271

**How many observations (codes for countries) are there in CountryMap?**

In [16]:
country_map.shape[0]

149

**How many interviewees have a missing value for the new metropolitan area variable? Note that all of these interviewees would have been removed from the merged data frame if we did not include the all.x=TRUE parameter.**

In [17]:
df['MetroArea'].isnull().sum()

34238

**Which metropolitan area has the highest proportion of interviewees of Hispanic ethnicity?**

In [18]:
df.groupby('MetroArea').agg({'Hispanic': 'mean'}).sort_values('Hispanic').tail(1)

,Hispanic
MetroArea,
"Laredo, TX",0.966292


**How many metropolitan areas in the US have at least 20% of interviewees that are Asian?**

In [19]:
df.groupby('MetroArea').agg({'Race': lambda x: np.mean(x == 'Asian')}).sort_values('Race').tail(4)

,Race
MetroArea,
"Vallejo-Fairfield, CA",0.203008
"San Jose-Sunnyvale-Santa Clara, CA",0.241791
"San Francisco-Oakland-Fremont, CA",0.246753
"Honolulu, HI",0.501904


**Answer:** 4

***

**Which metropolitan area has the smallest proportion of interviewees who have received no high school diploma?**

In [20]:
df.groupby('MetroArea').agg({'Education': lambda x: np.mean(x == 'No high school diploma')})\
                            .sort_values('Education').head(1)

,Education
MetroArea,
"Iowa City, IA",0.022901


**How may interviewees have a missing value for the new country of birth variable?**

In [21]:
df['Country'].isnull().sum()

176

**Among all interviewees born outside of North America, which country was the most common place of birth?**

In [22]:
df['Country'].value_counts().head(3)

United States    115063
Mexico             3921
Philippines         839
Name: Country, dtype: int64

**Answer:** Philippines

**What proportion of the interviewees from the "New York-Northern New Jersey-Long Island, NY-NJ-PA" metropolitan area have a country of birth that is not the United States? For this computation, don't include people from this metropolitan area who have a missing country of birth.**

In [23]:
np.mean(df[df.MetroArea == "New York-Northern New Jersey-Long Island, NY-NJ-PA"]['Country'] != 'United States')

0.30929931595489002

**Which metropolitan area has the largest number (note -- not proportion) of interviewees with a country of birth in India?**

In [24]:
df.groupby('MetroArea').agg({'Country' : lambda x: sum(x == 'India')}).sort_values('Country').tail(1)

,Country
MetroArea,
"New York-Northern New Jersey-Long Island, NY-NJ-PA",96
